# TEST 2:

In [9]:
# Add the 'src' as root folder, to find other modules in the project
import sys
sys.path.append("../")

In [10]:
# import other internal modules and functions
from simulator.swing_equation import swing_equation

# import other external modules and functions
from sbi.inference import SNPE, simulate_for_sbi, prepare_for_sbi
from sbi.utils import BoxUniform, posterior_nn
import torch
import os
from matplotlib import pyplot as plt
from scipy import stats
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew

In [4]:
# Define the experimental setup variables
num_simulations = 10000
timesteps = [0.1, 0.05, 0.01]

# Define the prior distribution for the parameters
prior = BoxUniform(
    low=torch.tensor([0.1, 1.0, 0.1, 40.0, 0.1]),
    high=torch.tensor([10.0, 100.0, 10.0, 70.0, 10.0])
)

# Define the true parameters for training and evaluation
true_parameters = torch.tensor([4.0, 40.0, 3.0, 60.0, 7.0])

posteriors = []

for dt in timesteps:

    # wrapper for the simulator, to allow different time durations
    def simulator(parameters):
        return swing_equation(parameters, dt=dt)
    
    # Simulate the observation
    observation = simulator(true_parameters)

    # Wrap the simulator function for compatibility with SBI
    simulator_wrapper, prior = prepare_for_sbi(simulator, prior)

    # Instantiate the neural density estimator
    neural_posterior = posterior_nn(
        model="maf", hidden_features=10, num_transforms=2
    )

    # Set up the inference procedure with the SNPE-C procedure
    inference = SNPE(prior=prior, density_estimator=neural_posterior)

    # Run the inference procedure to generate samples and corresponding simulated data points
    theta, x = simulate_for_sbi(simulator_wrapper, prior, num_simulations=num_simulations)

    # Train the neural density estimator
    density_estimator = inference.append_simulations(theta, x).train()

    # Build the posterior for the given parameters
    posterior = inference.build_posterior(density_estimator)
    posterior.set_default_x(observation)

    posteriors.append(posterior)

Running 10000 simulations.: 100%|██████████| 10000/10000 [00:53<00:00, 185.23it/s]


 Neural network successfully converged after 366 epochs.

Running 10000 simulations.: 100%|██████████| 10000/10000 [01:44<00:00, 96.03it/s]


 Neural network successfully converged after 228 epochs.

Running 10000 simulations.: 100%|██████████| 10000/10000 [08:20<00:00, 19.96it/s]


 Neural network successfully converged after 173 epochs.

In [8]:
# Define the directory path
dir_path = '../../results/simulation_lengths/plots/'

# Create the directories
os.makedirs(dir_path, exist_ok=True)

# Plot settings
param_names = ['c1', 'c2', 'P0', 'P1', 'epsilon']

for idx, t in enumerate(timesteps):
    
    # Extract posterior distributions for current T
    posterior_samples = posteriors[idx].sample((10000,), show_progress_bars=False)

    # Create separate figures for each type of plot
    fig1, axes1 = plt.subplots(1, len(param_names), figsize=(20, 4))
    fig2, axes2 = plt.subplots(1, len(param_names), figsize=(20, 4))
    fig3, axes3 = plt.subplots(1, len(param_names), figsize=(20, 4))

    # Plot each parameter for current T
    for j in range(len(param_names)):
        parameter_samples = posterior_samples[:, j]
        
        # Histogram
        axes1[j].hist(parameter_samples, bins=30, density=True, alpha=0.7, label='Inferred')
        axes1[j].axvline(x=true_parameters[j], color='red', linestyle='--', label='True')

        axes1[j].set_title(param_names[j] + ' - Histogram')
        axes1[j].legend()
        
        # Scatter plot
        axes2[j].scatter(range(len(parameter_samples)), parameter_samples, alpha=0.7)
        axes2[j].set_title(param_names[j] + ' - Scatter plot')
        
        # QQ plot
        stats.probplot(parameter_samples, dist="norm", plot=axes3[j])
        axes3[j].set_title(param_names[j] + ' - QQ plot')

    # Set titles for the figures
    fig1.suptitle(f'Posterior Distributions for dt={t} - Histograms', fontsize=16)
    fig2.suptitle(f'Posterior Distributions for dt={t} - Scatter Plots', fontsize=16)
    fig3.suptitle(f'Posterior Distributions for dt={t} - QQ Plots', fontsize=16)

    # Adjust layout and save the figures
    fig1.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig1.savefig(os.path.join(dir_path, f'histogram_dt_{t}.pdf'))
    fig2.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig2.savefig(os.path.join(dir_path, f'scatter_dt_{t}.pdf'))
    fig3.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig3.savefig(os.path.join(dir_path, f'qqplot_dt_{t}.pdf'))

    plt.show()


IndexError: list index out of range

In [5]:
# Initialize lists to store the results
medians = []
stds = []
kurtoses = []
skews = []

# Define the directory path
dir_path = '../../results/simulation_lengths/csv/'

# Create the directories
os.makedirs(dir_path, exist_ok=True)

# Calculate the statistics for each posterior
for idx, t in enumerate(timesteps):
    posterior = posteriors[idx]
    samples = posterior.sample((10000,), show_progress_bars=False)
    samples_np = samples.numpy()
    median = np.median(samples_np, axis=0)
    std = np.std(samples_np, axis=0)
    kurt = kurtosis(samples_np, axis=0)
    skewness = skew(samples_np, axis=0)
    
    medians.append([t] + list(median))
    stds.append([t] + list(std)) 
    kurtoses.append([t] + list(kurt))
    skews.append([t] + list(skewness))

# Create a DataFrame for each statistic
df_medians = pd.DataFrame(medians, columns=['T'] + param_names)
df_medians.set_index('T', inplace=True)

df_stds = pd.DataFrame(stds, columns=['T'] + param_names)
df_stds.set_index('T', inplace=True)

df_kurtoses = pd.DataFrame(kurtoses, columns=['T'] + param_names)
df_kurtoses.set_index('T', inplace=True)

df_skews = pd.DataFrame(skews, columns=['T'] + param_names)
df_skews.set_index('T', inplace=True)

# Save the DataFrames as CSV files in the specified directory
df_medians.to_csv(os.path.join(dir_path, 'medians.csv'))
df_stds.to_csv(os.path.join(dir_path, 'standard_deviations.csv'))
df_kurtoses.to_csv(os.path.join(dir_path, 'kurtoses.csv'))
df_skews.to_csv(os.path.join(dir_path, 'skews.csv'))

# Print the tables
print("Medians:")
print(df_medians)
print("\nStandard Deviations:")
print(df_stds)
print("\nKurtoses:")
print(df_kurtoses)
print("\nSkews:")
print(df_skews)


NameError: name 'timesteps' is not defined